In [42]:
import re
import time
import requests
import json
import pandas as pd
import numpy as np
import os
import torch
import os
import sys
sys.path.insert(0, os.path.abspath('../')) # needed to import src
from src.existing_models.twitteraae.code.predict import Predict
from sklearn.model_selection import train_test_split
OUTPUT_DIRECTORY = '../data'

In [9]:
'''
Davidson:
count = number of CrowdFlower users who coded each tweet (min is 3, sometimes more users coded a tweet when judgments were determined to be unreliable by CF).

hate_speech = number of CF users who judged the tweet to be hate speech.
offensive_language = number of CF users who judged the tweet to be offensive.
neither = number of CF users who judged the tweet to be neither offensive nor non-offensive.
class = class label for majority of CF users. 0 - hate speech 1 - offensive language 2 - neither

'''
def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    #parsed_text = parsed_text.code("utf-8", errors='ignore')
    return parsed_text

In [11]:

raw_data = pd.read_csv(f'{OUTPUT_DIRECTORY}/twitter_datasets/davidson_labelled.csv', index_col=0)
raw_data['cleaned_tweet'] = raw_data.apply(lambda row: preprocess(row['tweet']), axis=1)
raw_data['is_hate'] = np.where(raw_data['class'] == 0, 1, 0)
raw_data['is_offensive'] = np.where(raw_data['class'] == 1, 1, 0)

,count,hate_speech,offensive_language,neither,class,tweet,cleaned_tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,!!! RT : As a woman you shouldn't complain abo...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,!!!!! RT : boy dats cold...tyga dwn bad for cu...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,!!!!!!! RT Dawg!!!! RT : You ever fuck a bitc...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,!!!!!!!!! RT : she look like a tranny
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,!!!!!!!!!!!!! RT : The shit you hear about me ...
...,...,...,...,...,...,...,...
25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,you's a muthaf***in lie &#8220;: right! His ...
25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an...","you've gone and broke the wrong heart baby, an..."
25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...,young buck wanna eat!!.. dat nigguh like I ain...
25295,6,0,6,0,1,youu got wild bitches tellin you lies,youu got wild bitches tellin you lies


In [ ]:
'''
Waseem 1 & 2:

- Waseem & Hovy (2016) DS contains sampling bias -  70% of sexist tweets were from two users, 99% of racist tweets were from one user
- In Sap et al this leads to higher false positive rates of AAE dialect tweets
'''


In [ ]:
'''
Golbeck et al:
- literature link: http://www.cs.umd.edu/~golbeck/papers/trolling.pdf
- label distribution: 5,495 positive examples and 29,505 negative example
- need to request access via:  jgolbeck@umd.edu

 ID is a unique identifier. 
 Code is the label - H for Harassing and N for not-harassment. Please read the paper for details!
 Tweet is the text of the tweet.
 
 There is duplicate text tweets, may want to remove. 
'''


In [21]:
'''
Founta et al:
80k 
- literature link: https://datalab.csd.auth.gr/wp-content/uploads/publications/17909-77948-1-PB.pdf
- dataset Link: https://dataverse.mpi-sws.org/dataset.xhtml?persistentId=doi:10.5072/FK2/ZDTEMN
'abusive', 'normal', 'hateful', 'spam' & nan as maj_label
'''

founta = pd.read_csv(f'{OUTPUT_DIRECTORY}/twitter_datasets/founta/hatespeechtwitter.csv')
founta.dropna(inplace=True) # 4 instances
founta['is_hate'] = np.where(founta['maj_label'] == 'hateful', 1, 0)
founta['is_abusive'] = np.where(founta['maj_label'] == 'abusive', 1, 0)
founta

,tweet_id,maj_label,is_hate,is_abusive
0,849667487180259329,abusive,0,1
1,850490912954351616,abusive,0,1
2,848791766853668864,abusive,0,1
3,848306464892604416,abusive,0,1
4,850010509969465344,normal,0,0
...,...,...,...,...
79995,847640895956459521,normal,0,0
79996,850335795022102530,normal,0,0
79997,848201162717265920,normal,0,0
79998,848939985160077312,spam,0,0


In [39]:
 #can do 100 at atime, 300 requests/15 mins
def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)



jprint(response)
# twitter_api = twitter.Api(consumer_key='AJXjzeRH380K0uJV5Rlw8iakN',
#                   consumer_secret='Dq1GgRWxZfyJEQfRuRfHtzuZWHmmOJpNQIRqs2pi9nMKla8ng4',
#                   access_token_key='AAAAAAAAAAAAAAAAAAAAAI7JIwEAAAAAUI90MzpQE%2BPgKDyGmP%2FCladtxxU%3D06inEDvISc2MgMTrR7l1Zk5cOvHnW6JqysPlv9sFvoePsZNPAu',
#                   access_token_secret=[access token secret])
# def get_tweet(tweet_id):
#     pass

##%%

{
    "data": [
        {
            "id": "850490912954351616",
            "text": "Alex Brosas another idiot #ALDUBKSGoesToUS  https://t.co/14G7hFwVQm"
        },
        {
            "id": "850335795022102530",
            "text": "#LaserHairRemoval for permanently smooth skin! Free Consultation 781-599-0666 https://t.co/G9q2FKIdkE"
        },
        {
            "id": "847640895956459521",
            "text": "I liked a @YouTube video https://t.co/SKxJS9FlxX How Future\u2019s \u201cMask Off\u201d Made The Flute Hip-Hop\u2019s Hottest Instrument | Genius"
        }
    ],
    "errors": [
        {
            "detail": "Could not find tweet with ids: [849667487180259329].",
            "parameter": "ids",
            "resource_type": "tweet",
            "title": "Not Found Error",
            "type": "https://api.twitter.com/2/problems/resource-not-found",
            "value": "849667487180259329"
        }
    ]
}


[{'id': '850490912954351616',
  'text': 'Alex Brosas another idiot #ALDUBKSGoesToUS  https://t.co/14G7hFwVQm'},
 {'id': '850335795022102530',
  'text': '#LaserHairRemoval for permanently smooth skin! Free Consultation 781-599-0666 https://t.co/G9q2FKIdkE'},
 {'id': '847640895956459521',
  'text': 'I liked a @YouTube video https://t.co/SKxJS9FlxX How Future’s “Mask Off” Made The Flute Hip-Hop’s Hottest Instrument | Genius'},
 {'id': '847640895956459521',
  'text': 'I liked a @YouTube video https://t.co/SKxJS9FlxX How Future’s “Mask Off” Made The Flute Hip-Hop’s Hottest Instrument | Genius'}]

In [55]:
bearer_token = "AAAAAAAAAAAAAAAAAAAAAI7JIwEAAAAAUI90MzpQE%2BPgKDyGmP%2FCladtxxU%3D06inEDvISc2MgMTrR7l1Zk5cOvHnW6JqysPlv9sFvoePsZNPAu"
headers = {"Authorization": f"Bearer {bearer_token}"}
def get_tweets(tweet_ids: str) -> json:
    params={'ids': tweet_ids}
    return requests.get(url='https://api.twitter.com/2/tweets', params=params, headers=headers).json()

In [71]:
count = 0
last_start = 0
tweets_founta = list()
for i in range(100, founta.shape[0], 100):
    if count == 299:
        print('Pausing, reached 300 requests')
        time.sleep(900) #wait 15 mins to not overgo limit
        count = 0
    converted_list = [str(id) for id in founta[last_start:i]['tweet_id'].values]
    ids = ",".join(converted_list)
    response = get_tweets(ids)
    # jprint(response)
    tweets_founta.extend(response['data'])
    last_start=i
    count += 1

Pausing, reached 300 requests
Pausing, reached 300 requests


In [90]:
founta_tweets_pd = pd.DataFrame(tweets_founta)
founta_tweets_pd['id'] = founta_tweets_pd['id'].astype('int64')

In [91]:
founta_tweets_pd

,id,text,withheld
0,850490912954351616,Alex Brosas another idiot #ALDUBKSGoesToUS ht...,NaN
1,848791766853668864,"RT @ItIzBiz: as Nancy Reagan would say, 'just ...",NaN
2,850010509969465344,RT @MailOnline: The Nazi death gas so horrific...,NaN
3,850433664890544128,I hate er chase because if the Bitch that work...,NaN
4,850100742035836929,RT @nyctophil3: Pineapples do not belong on pi...,NaN
...,...,...,...
50393,850773885847056385,There's a stop and search underway on the A316...,NaN
50394,848365063517982720,"@gerbsch78 And now, his singing of that Seal s...",NaN
50395,848569917527994370,I suppose you'll turn down foreign investment ...,NaN
50396,848592994567471105,It's your turn to Enter the Stars.\n\nWe are g...,NaN


In [92]:
founta_twitter = founta.merge(founta_tweets_pd[['id','text']],left_on='tweet_id', right_on='id', how='left') #79996 rows originally, only able to retrieve 50398 tweets
founta_twitter

,tweet_id,maj_label,is_hate,is_abusive,id,text
0,849667487180259329,abusive,0,1,NaN,NaN
1,850490912954351616,abusive,0,1,8.504909e+17,Alex Brosas another idiot #ALDUBKSGoesToUS ht...
2,848791766853668864,abusive,0,1,8.487918e+17,"RT @ItIzBiz: as Nancy Reagan would say, 'just ..."
3,848306464892604416,abusive,0,1,NaN,NaN
4,850010509969465344,normal,0,0,8.500105e+17,RT @MailOnline: The Nazi death gas so horrific...
...,...,...,...,...,...,...
80079,847640895956459521,normal,0,0,NaN,NaN
80080,850335795022102530,normal,0,0,NaN,NaN
80081,848201162717265920,normal,0,0,NaN,NaN
80082,848939985160077312,spam,0,0,NaN,NaN


In [88]:

founta_tweets_pd['text'] = founta_tweets_pd['text'].astype('str')
founta_tweets_pd[['id','text']].dtypes

id       int64
text    object
dtype: object

In [98]:
founta_twitter.dropna(inplace=True)
founta_twitter.drop(columns=['id'])
founta_twitter.to_csv(f'{OUTPUT_DIRECTORY}/twitter_datasets/founta/with_tweets.csv')

In [97]:
founta_twitter['cleaned_tweet'] = raw_data.apply(lambda row: preprocess(row['text']), axis=1)
founta_twitter

KeyError: 'text'